In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime 
import seaborn as sb
import scipy.stats as stat
%matplotlib inline
import random

In [2]:
bikeshare = pd.read_csv('bikeshare_reduced.csv', 
                        parse_dates=['Start date', 'End date', 'start_date_short', 'end_date_short'])
bikeshare.drop('Unnamed: 0', 1, inplace=True)

lat_lon = pd.read_csv('bikeshare_locations.csv')
print(lat_lon.head())

    OBJECTID  ID                                     ADDRESS  TERMINAL_NUMBER  \
0  173869668   1                         Eads St & 15th St S            31000   
1  173869669   2                             18th & Eads St.            31001   
2  173869670   3                           20th & Crystal Dr            31002   
3  173869671   4                           15th & Crystal Dr            31003   
4  173869672   5  Aurora Hills Community Ctr/18th & Hayes St            31004   

    LATITUDE  LONGITUDE INSTALLED LOCKED  INSTALL_DATE  REMOVAL_DATE  \
0  38.858971 -77.053230       YES     NO           NaN           NaN   
1  38.857250 -77.053320       YES     NO           NaN           NaN   
2  38.856425 -77.049232       YES     NO           NaN           NaN   
3  38.860170 -77.049593       YES     NO           NaN           NaN   
4  38.857866 -77.059490       YES     NO           NaN           NaN   

  TEMPORARY_INSTALL  NUMBER_OF_BIKES  NUMBER_OF_EMPTY_DOCKS            X  \
0   

In [3]:
lat_lon = lat_lon[['ADDRESS', 'LATITUDE', 'LONGITUDE', 'TERMINAL_NUMBER']]

In [4]:
lat_lon.head()

,ADDRESS,LATITUDE,LONGITUDE,TERMINAL_NUMBER
0,Eads St & 15th St S,38.858971,-77.053230,31000
1,18th & Eads St.,38.857250,-77.053320,31001
2,20th & Crystal Dr,38.856425,-77.049232,31002
3,15th & Crystal Dr,38.860170,-77.049593,31003
4,Aurora Hills Community Ctr/18th & Hayes St,38.857866,-77.059490,31004


In [5]:
bikeshare.head()

,Start date,End date,Start station,End station,Member Type,start_date_short,end_date_short,time_diff,season,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed
0,2011-06-30 23:59:00,2011-07-01 00:09:00,11th & Kenyon St NW (31102),Calvert St & Woodley Pl NW (31106),Registered,2011-06-30,2011-07-01,10.0,3,6,0,4,1,1,0.696667,0.637008,0.434167,0.185312
1,2011-06-30 23:58:00,2011-07-01 00:03:00,Lamont & Mt Pleasant NW (31107),Adams Mill & Columbia Rd NW (31104),Registered,2011-06-30,2011-07-01,5.0,3,6,0,4,1,1,0.696667,0.637008,0.434167,0.185312
2,2011-06-30 23:59:00,2011-07-01 00:09:00,Massachusetts Ave & Dupont Circle NW (31200),Adams Mill & Columbia Rd NW (31104),Casual,2011-06-30,2011-07-01,10.0,3,6,0,4,1,1,0.696667,0.637008,0.434167,0.185312
3,2011-06-30 23:58:00,2011-07-01 00:17:00,4th St & Massachusetts Ave NW (31604),C & O Canal & Wisconsin Ave NW (31225),Registered,2011-06-30,2011-07-01,19.0,3,6,0,4,1,1,0.696667,0.637008,0.434167,0.185312
4,2011-06-30 23:58:00,2011-07-01 00:28:00,10th St & Constitution Ave NW (31219),14th & D St NW / Ronald Reagan Building (31231),Casual,2011-06-30,2011-07-01,30.0,3,6,0,4,1,1,0.696667,0.637008,0.434167,0.185312


In [85]:
small = bikeshare.copy()

In [86]:
small['start_station'], small['start_station_code'] = small['Start station'].str.split("(", 1).str

In [87]:
small['start_station_code'] = small['start_station_code'].str.replace(')', '')

In [89]:
sum(small['start_station_code'].isnull())

354030

In [90]:
len(small['Start station'].unique())

294

In [91]:
len(lat_lon)

485

In [92]:
small['end_station'], small['end_station_code'] = small['End station'].str.split("(", 1).str

In [94]:
small['end_station_code'] = small['end_station_code'].str.replace(')', '')

In [95]:
small.head()

,Start date,End date,Start station,End station,Member Type,start_date_short,end_date_short,time_diff,season,mnth,...,workingday,weathersit,temp,atemp,hum,windspeed,start_station,start_station_code,end_station,end_station_code
0,2011-06-30 23:59:00,2011-07-01 00:09:00,11th & Kenyon St NW (31102),Calvert St & Woodley Pl NW (31106),Registered,2011-06-30,2011-07-01,10.0,3,6,...,1,1,0.696667,0.637008,0.434167,0.185312,11th & Kenyon St NW,31102,Calvert St & Woodley Pl NW,31106
1,2011-06-30 23:58:00,2011-07-01 00:03:00,Lamont & Mt Pleasant NW (31107),Adams Mill & Columbia Rd NW (31104),Registered,2011-06-30,2011-07-01,5.0,3,6,...,1,1,0.696667,0.637008,0.434167,0.185312,Lamont & Mt Pleasant NW,31107,Adams Mill & Columbia Rd NW,31104
2,2011-06-30 23:59:00,2011-07-01 00:09:00,Massachusetts Ave & Dupont Circle NW (31200),Adams Mill & Columbia Rd NW (31104),Casual,2011-06-30,2011-07-01,10.0,3,6,...,1,1,0.696667,0.637008,0.434167,0.185312,Massachusetts Ave & Dupont Circle NW,31200,Adams Mill & Columbia Rd NW,31104
3,2011-06-30 23:58:00,2011-07-01 00:17:00,4th St & Massachusetts Ave NW (31604),C & O Canal & Wisconsin Ave NW (31225),Registered,2011-06-30,2011-07-01,19.0,3,6,...,1,1,0.696667,0.637008,0.434167,0.185312,4th St & Massachusetts Ave NW,31604,C & O Canal & Wisconsin Ave NW,31225
4,2011-06-30 23:58:00,2011-07-01 00:28:00,10th St & Constitution Ave NW (31219),14th & D St NW / Ronald Reagan Building (31231),Casual,2011-06-30,2011-07-01,30.0,3,6,...,1,1,0.696667,0.637008,0.434167,0.185312,10th St & Constitution Ave NW,31219,14th & D St NW / Ronald Reagan Building,31231
